In [1]:
%matplotlib ipympl

In [2]:
# initialization
from rayoptics.environment import *
from rayoptics.raytr import trace

In [3]:
from numpy.linalg import norm
from scipy.optimize import minimize_scalar, bracket, bisect, fsolve, newton

In [4]:
root_pth = Path(rayoptics.__file__).resolve().parent

In [5]:
app = AppManager(None)

# Create a new model

In [6]:
app.model = open_model(root_pth/"codev/tests/ag_dblgauss.seq")
opm = app.model
sm  = opm['seq_model']
osp = opm['optical_spec']
pm = opm['parax_model']
em = opm['ele_model']
pt = opm['part_tree']
ar = opm['analysis_results']
print(f"{opm.ro_version=}")

opm.ro_version='0.9.5'


In [7]:
opm.update_model()

In [8]:
sm.list_model()

              c            t        medium     mode   zdr      sd
  Obj:     0.000000  9.93938e+11       air             1  2.4782e+11
    1:     0.017793      8.75000    N-SSK2             1      28.172
    2:     0.006567     0.500000       air             1      27.136
    3:     0.026537      12.5000     N-SK2             1      24.626
    4:     0.000000      3.80000        F5             1      22.894
    5:     0.041269      16.3694       air             1      17.293
 Stop:     0.000000      13.7480       air             1      15.758
    7:    -0.035239      3.80000        F5             1      15.135
    8:     0.000000      11.0000    N-SK16             1      17.250
    9:    -0.026368     0.500000       air             1      18.926
   10:     0.005637      7.00000    N-SK16             1      20.349
   11:    -0.012593      61.4875       air             1      20.776
  Img:     0.000000      0.00000                       1      24.739


In [9]:
stop = sm.stop_surface

In [10]:
def y_stop_coordinate(y1, *args):
    seq_model, ifcx, pt0, dist, wvl, target = args
    pt1 = np.array([0., y1, dist])
    dir0 = pt1 - pt0
    length = norm(dir0)
    dir0 = dir0/length
    ray, _, _ = rt.trace(seq_model, pt0, dir0, wvl)
    y_ray = ray[ifcx][mc.p][1]
    print(y1, y_ray)
    return y_ray - target

In [11]:
def surface_coordinate(coord, *args):
    seq_model, ifcx, pt0, dist, wvl, target = args
    pt1 = np.array([coord[0], coord[1], dist])
    dir0 = pt1 - pt0
    length = norm(dir0)
    dir0 = dir0/length
    ray, _, _ = rt.trace(seq_model, pt0, dir0, wvl)
    xy_ray = np.array([ray[ifcx][mc.p][0], ray[ifcx][mc.p][1]])
    print(coord[0], coord[1], xy_ray[0], xy_ray[1])
    return xy_ray - target

In [12]:
fld, wvl, foc = osp.lookup_fld_wvl_focus(2)

In [13]:
fod = ar['parax_data'].fod
dist = fod.obj_dist + fod.enp_dist
pt0 = osp.obj_coords(fld)

In [14]:
#res = newton(y_stop_coordinate, 0.)
y_target = 0.
start_y = newton(y_stop_coordinate, 0.,
                 args=(sm, stop, pt0, dist, wvl, y_target))

/opt/miniconda3/envs/ro_usr_312/lib/python3.12/site-packages/rayoptics/elem/profiles.py:436: SyntaxWarning: invalid escape sequence '\s'
  """ Conic surface profile parameterized by curvature and conic constant.
/opt/miniconda3/envs/ro_usr_312/lib/python3.12/site-packages/rayoptics/elem/profiles.py:659: SyntaxWarning: invalid escape sequence '\s'
  """ Even Polynomial asphere, even terms up to 20th order, on base conic.
/opt/miniconda3/envs/ro_usr_312/lib/python3.12/site-packages/rayoptics/elem/profiles.py:868: SyntaxWarning: invalid escape sequence '\s'
  """ Radial Polynomial asphere, both even and odd terms, on base conic.
/opt/miniconda3/envs/ro_usr_312/lib/python3.12/site-packages/rayoptics/elem/profiles.py:1095: SyntaxWarning: invalid escape sequence '\s'
  """Y Aspheric toroid, even terms up to 20th order, on base conic.
/opt/miniconda3/envs/ro_usr_312/lib/python3.12/site-packages/rayoptics/elem/profiles.py:1351: SyntaxWarning: invalid escape sequence '\s'
  """X Aspheric toroid

AttributeError: 'tuple' object has no attribute 'dot'

In [ ]:
#res = fsolve(surface_coordinate, np.array([0., 0.]),
#             args=(sm, sm.stop_surface, sm.central_wavelength(), np.array([0., 0.])))
xy_target = np.array([0., 0.])
start_coords = fsolve(surface_coordinate, np.array([0., 0.]),
                      epsfcn=0.0001*fod.enp_radius,
                      args=(sm, stop, pt0, dist, wvl, xy_target))

In [ ]:
start_coords

In [ ]:
aim_pt = trace.iterate_ray(opm, sm.stop_surface, np.array([0., 0.]), fld, wvl)

In [ ]:
print(start_coords, aim_pt, start_coords[1]-aim_pt[1])